<img src="https://drive.google.com/uc?export=view&id=1Q6vQcIWFPY27isBepABpJ7nroUNKox_Z" width="100%">

# **Taller 4**
---

En este taller se evaluarán las habilidades adquiridas en aprendizaje supervisado a partir del conjunto de datos [reseñas de películas en español de FilmAffinity](https://www.kaggle.com/datasets/andrsmosquera/crticas-pelculas-filmaffinity-en-espaol-netflix).

El conjunto de datos de reseñas de películas de FilmAffinity es un conjunto de datos de reseñas de películas en español que se utiliza para entrenar y evaluar modelos de clasificación de sentimiento. El conjunto de datos consta de reseñas de películas en español, clasificadas según la calificación del usuario (por ejemplo, positivas o negativas).

Las reseñas de películas incluidas en el conjunto de datos son reseñas de películas de la plataforma FilmAffinity. Las reseñas son de diferentes géneros de películas (comedia, terror, acción, etc.).

El conjunto de datos es anotado manualmente y puede considerarse un conjunto de datos desbalanceado, dependiendo de cómo se defina la clasificación (por ejemplo, reseñas positivas vs. negativas). El conjunto de datos es muy utilizado para entrenar y evaluar modelos de clasificación de sentimiento en español.

Comenzaremos importando las librerías necesarias:

In [ ]:
#TEST_CELL
!pip install unidecode

In [ ]:
import re
import spacy
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from unidecode import unidecode
from IPython.display import display

Ahora cargamos el conjunto de datos:

In [ ]:
#TEST_CELL
!wget https://raw.githubusercontent.com/mindlab-unal/mlds4-case-study/refs/heads/main/dataset/film_reviews_result.csv

In [ ]:
#TEST_CELL
data = pd.read_csv("film_reviews_result.csv", sep="|")

display(data.head())

Como podemos ver, el conjunto tiene columnas:

- `film_name`: Título de la película.
- `gender`: Género de la película (comedia, terror, acción, etc.).
- `film_avg_rate`: Nota media de la película (votos de todos los usuarios).
- `review_rate`: Nota que el usuario que hace la crítica pone a la película.
- `review_title`: Título de la crítica.
- `review_text`: Crítica de la película.

Para nuestro taller, nos centraremos en las columnas:

- `review_text`: Texto de la reseña de la película.
- `review_rate`: Calificación de la reseña, que utilizaremos para crear nuestras etiquetas de clasificación."

Vamos a preprocesar el conjunto de datos:

In [ ]:
nlp = spacy.blank("en")
def preprocess(text):
    doc = nlp(text) # creamos un documento de spacy
    no_stops = " ".join(
        token.text
        for token in filter(
            lambda token: not token.is_stop and len(token) > 3 and len(token) < 24,
            doc,
            )
        ) # eliminamos stopwords y palabras por longitud
    norm_text = unidecode(no_stops.lower()) # normalizamos el texto
    no_chars = re.sub(r"[^a-z ]", " ", norm_text) # eliminamos caracteres especiales
    no_spaces = re.sub(r"\s+", " ", no_chars) # eliminamos espacios duplicados
    return no_spaces.strip()

Aplicamos la función de preprocesamiento:

In [ ]:
data["corpus"] = data.review_text.apply(preprocess)

Inspeccionemos el tamaño de este conjunto de datos:

In [ ]:
#TEST_CELL
display(data.shape)

También podemos ver la distribución de etiquetas:

In [ ]:
#TEST_CELL
fig, ax = plt.subplots()
labels, counts = np.unique(data["review_rate"], return_counts=True)
ax.bar(labels, counts)
ax.set_xlabel("Categoría")
ax.set_ylabel("Conteo")
fig.show()

Como podemos ver, se trata de un conjunto desbalanceado.

## **1. Extracción de Características**
---

En este punto, deberás **codificar numéricamente el corpus** utilizando la técnica **TF-IDF**. Para ello, deberás entrenar un **vectorizador TF-IDF** con **sublinear scaling** y restringir el vocabulario a los **1,000 términos más frecuentes** del conjunto de datos.

Deberás implementar la función **`vectorizer`**, la cual recibirá como entrada el corpus preprocesado y deberá retornar tanto la representación vectorial como el vectorizador entrenado.

### **Parámetros**

- `corpus`: Objeto de tipo `pd.Series` que contiene los textos preprocesados.

### **Retorna**

- `features`: Arreglo de tipo `numpy` que contiene la representación TF-IDF del corpus.
- `vect`: Objeto `TfidfVectorizer` entrenado con las especificaciones indicadas.


<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pistas</b></font>
</summary>

- Recuerde que _sublinear scaling_ se puede controlar con el parámetro `sublinear_tf` del vectorizador.
- Recuerde convertir la representación en un arreglo de `numpy`.
</details>

In [ ]:
# FUNCIÓN CALIFICADA vectorizer:
from sklearn.feature_extraction.text import TfidfVectorizer

def vectorizer(corpus):
    ### ESCRIBA SU CÓDIGO AQUÍ ###
    vect = ...
    features = ...
    return features, vect
    ### FIN DEL CÓDIGO ###

In [ ]:
#TEST_CELL
features, vect = vectorizer(data.corpus)
display(features.shape)

**Salida esperada**:

En este primer ejemplo debe obtener el tamaño de la representación:

```python
❱ display(features.shape)
(10058, 1000)
```

In [ ]:
#TEST_CELL
features, vect = vectorizer(data.corpus)
display(vect.get_feature_names_out()[:5])

**Salida esperada**:

En este caso deberá obtener las primeras 5 palabras del vocabulario:

```python
❱ display(vect.get_feature_names_out()[:5])
array(['absolutamente', 'absoluto', 'absurdo', 'aburrida', 'acaba'],
      dtype=object)
```

In [ ]:
#TEST_CELL
features, vect = vectorizer(data.corpus)
print(features.sum())

**Salida esperada**:

En este caso deberá obtener la suma de toda la matriz:

```python
❱ print(features.sum())
69922.60724691708
```

In [ ]:
import re
import spacy
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from unidecode import unidecode

# datos
data = pd.read_csv("film_reviews_result.csv", sep="|")

data["corpus"] = data.review_text.apply(preprocess)

#comparar textos

def compare_texts(expected, result):
  if expected == result:
    return True
  else:
    return False

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
features, vect = vectorizer(data.corpus)

In [ ]:
expected = 69924.10844646816
exp_features = ['absolutamente', 'absoluto', 'absurdo', 'aburrida', 'acaba']
exp_shape = (10058, 1000)
exp_max_features = 1000
result = features.sum()
print(isinstance(features, np.ndarray), isinstance(vect,TfidfVectorizer), vect.sublinear_tf, np.isclose(expected,result, atol=2), compare_texts(exp_features, list(vect.get_feature_names_out()[:5])),compare_texts(exp_shape, features.shape), compare_texts(exp_max_features, vect.max_features))

In [ ]:
expected = 69922.60724691708
exp_features = ['absolutamente', 'absoluto', 'absurdo', 'aburrida', 'acaba']
exp_shape = (10058, 1000)
exp_max_features = 1000
result = features.sum()
print(isinstance(features, np.ndarray), isinstance(vect,TfidfVectorizer), vect.sublinear_tf, np.isclose(expected,result), compare_texts(exp_features, list(vect.get_feature_names_out()[:5])),compare_texts(exp_shape, features.shape), compare_texts(exp_max_features, vect.max_features))

In [ ]:
exp_features = ['absolutamente', 'absoluto', 'absurdo', 'aburrida', 'acaba']
exp_shape = (10058, 1000)
exp_max_features = 1000
print(compare_texts(exp_features, list(vect.get_feature_names_out()[:5])),compare_texts(exp_shape, features.shape), compare_texts(exp_max_features, vect.max_features))

## **2. Codificación de Etiquetas**
---

Para poder entrenar un modelo de aprendizaje automático, es necesario **codificar numéricamente las etiquetas**. En este caso, se debe implementar una solución sistemática utilizando el codificador **`LabelEncoder`** de la biblioteca **`sklearn`**.

Deberás implementar la función **`label_encode`**, la cual recibe como entrada una serie de etiquetas y retorna tanto la codificación numérica como el codificador entrenado.

### **Parámetros**

- `labels`: Objeto de tipo `pd.Series` que contiene las etiquetas en formato string.

### **Retorna**

- `encoded_labels`: Arreglo con las etiquetas codificadas de forma numérica.
- `encoder`: Objeto `LabelEncoder` entrenado.


<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pistas</b></font>
</summary>

- Recuerde invocar el método `.fit` del codificador antes de retornarlo.
- Evite usar el método `.fit_transform` para poder guardar el codificador.
</details>

In [ ]:
# FUNCIÓN CALIFICADA label_encode:
from sklearn.preprocessing import LabelEncoder

def label_encode(labels):
    ### ESCRIBA SU CÓDIGO AQUÍ ###
    encoder = ...
    encoded_labels = ...
    return encoded_labels, encoder
    ### FIN DEL CÓDIGO ###

In [ ]:
#TEST_CELL
encoded_labels, encoder = label_encode(data['review_rate'])
display(encoded_labels.shape)

**Salida esperada**:

En este primer ejemplo debe obtener el tamaño de las etiquetas:

```python
❱ display(encoded_labels.shape)
(10058,)
```

In [ ]:
#TEST_CELL
encoded_labels, encoder = label_encode(data['review_rate'])
print(encoded_labels.mean())

**Salida esperada**:

En este segundo ejemplo debe obtener el promedio de las etiquetas:

```python
❱ print(encoded_labels.mean())
4.852455756611652
```

In [ ]:
#TEST_CELL
encoded_labels, encoder = label_encode(data['review_rate'])
display(encoder.classes_)

**Salida esperada**:

En este caso debe obtener un arreglo con las posibles categorías y la posición a las que son asignadas:

```python
❱ display(encoder.classes_)
array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])
```

## **3. Particionamiento de Datos**
---

En esta etapa, deberás **dividir el conjunto de datos** en dos subconjuntos: uno para entrenamiento y otro para evaluación. Esta división debe realizarse de forma **estratificada**, es decir, manteniendo la proporción original de clases en ambos subconjuntos.

Para ello, implementa la función **`split_data`**, la cual recibe como entrada las características, las etiquetas, la proporción destinada a evaluación y una semilla para garantizar reproducibilidad. La función debe retornar los datos ya particionados.

### **Parámetros**

- `features`: Arreglo de tipo `numpy` que contiene la representación vectorial de los textos.
- `labels`: Arreglo de tipo `numpy` con las etiquetas codificadas.
- `test_size`: Proporción del conjunto de datos que se utilizará para evaluación (por ejemplo, `0.2` para un 20%).
- `seed`: Semilla de números aleatorios para garantizar resultados reproducibles.

### **Retorna**

- `features_train`: Subconjunto de características para entrenamiento.
- `features_test`: Subconjunto de características para evaluación.
- `labels_train`: Subconjunto de etiquetas para entrenamiento.
- `labels_test`: Subconjunto de etiquetas para evaluación.


<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pistas</b></font>
</summary>

- Recuerde usar el parámetro `stratify` para realizar la estratificación.
- `sklearn` maneja las semillas de números aleatorios con el parámetro `random_state`.
</details>

In [ ]:
# FUNCIÓN CALIFICADA split_data:
from sklearn.model_selection import train_test_split

def split_data(features, labels, test_size, seed):
    ### ESCRIBA SU CÓDIGO AQUÍ ###
    features_train = ...
    features_test = ...
    labels_train = ...
    labels_test = ...
    return features_train, features_test, labels_train, labels_test
    ### FIN DEL CÓDIGO ###

In [ ]:
#TEST_CELL
features, vect = vectorizer(data.corpus)
encoded_labels, encoder = label_encode(data['review_rate'])
features_train, features_test, labels_train, labels_test = split_data(
        features, encoded_labels, test_size=0.3, seed=42
        )
display(features_train.shape)
display(features_test.shape)
display(labels_train.shape)
display(labels_test.shape)

**Salida esperada**:

En este primer ejemplo debe obtener el tamaño de cada arreglo:

```python
❱ display(features_train.shape)
(7040, 1000)

❱ display(features_test.shape)
(3018, 1000)

❱ display(labels_train.shape)
(7040,)

❱ display(labels_test.shape)
(3018,)
```

In [ ]:
#TEST_CELL
features, vect = vectorizer(data.corpus)
encoded_labels, encoder = label_encode(data['review_rate'])
features_train, features_test, labels_train, labels_test = split_data(
        features, encoded_labels, test_size=0.3, seed=42
        )
print(labels_train.mean())
print(labels_test.mean())

**Salida esperada**:

En este segundo ejemplo debe obtener el promedio de las etiquetas en cada partición:

```python
❱ display(labels_train.mean())
4.853125

❱ display(labels_test.mean())
4.850894632206759
```

## **4. Modelo de Bosques Aleatorios**
---

En este punto, deberás **entrenar un modelo de Bosques Aleatorios (Random Forest)** utilizando los datos de entrenamiento.

Para ello, implementa la función **`random_forest`**, la cual recibe como entrada los datos de entrenamiento, la profundidad máxima de los árboles, el número de estimadores y una semilla para asegurar reproducibilidad. La función debe retornar el modelo ya entrenado.

### **Parámetros**

- `features_train`: Arreglo con las características del conjunto de entrenamiento.
- `labels_train`: Arreglo con las etiquetas correspondientes al conjunto de entrenamiento.
- `max_depth`: Profundidad máxima permitida para los árboles del modelo.
- `n_estimators`: Número total de árboles (estimadores) que compondrán el bosque.
- `seed`: Semilla de números aleatorios para asegurar resultados reproducibles.

### **Retorna**

- `model`: Objeto del tipo `RandomForestClassifier` ya entrenado con los parámetros especificados.


<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pistas</b></font>
</summary>

- Recuerde utilizar el método `fit` del modelo antes de retornarlo.
</details>

In [ ]:
# FUNCIÓN CALIFICADA random_forest:
from sklearn.ensemble import RandomForestClassifier

def random_forest(features_train, labels_train, max_depth, n_estimators, seed):
    ### ESCRIBA SU CÓDIGO AQUÍ ###
    model = ...
    return model
    ### FIN DEL CÓDIGO ###

In [ ]:
#TEST_CELL
features, vect = vectorizer(data.corpus)
encoded_labels, encoder = label_encode(data['review_rate'])
features_train, features_test, labels_train, labels_test = split_data(
        features, encoded_labels, test_size=0.3, seed=42
        )
model = random_forest(features_train, labels_train, 5, 50, 42)
display(model.max_depth)

**Salida esperada**:

En este primer ejemplo debe obtener la profundidad máxima del modelo:

```python
❱ display(model.max_depth)
5
```

In [ ]:
#TEST_CELL
features, vect = vectorizer(data.corpus)
encoded_labels, encoder = label_encode(data['review_rate'])
features_train, features_test, labels_train, labels_test = split_data(
        features, encoded_labels, test_size=0.3, seed=42
        )
model = random_forest(features_train, labels_train, 5, 50, 42)
display(model.n_estimators)

**Salida esperada**:

En este segundo ejemplo debe obtener el número de estimadores en el modelo:

```python
❱ display(model.n_estimators)
50
```

## **5. Evaluación**
---

En esta etapa, deberás **evaluar el desempeño del modelo entrenado** utilizando métricas estándar de clasificación. El objetivo es calcular las siguientes métricas para cada clase:

- Exactitud (*Accuracy*)
- Precisión (*Precision*)
- Sensibilidad o exhaustividad (*Recall*)
- Puntuación F1 (*F1-score*)

Para ello, implementa la función **`evaluation`**, que recibe como entrada el modelo entrenado junto con los datos de evaluación, y retorna un **reporte de clasificación** generado mediante `sklearn`.

### **Parámetros**

- `model`: Modelo previamente entrenado.
- `features_test`: Arreglo con las características del conjunto de evaluación.
- `labels_test`: Arreglo con las etiquetas verdaderas del conjunto de evaluación.

### **Retorna**

- `report`: Objeto tipo string generado por `classification_report` de `sklearn`, que resume las métricas por clase.


<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pistas</b></font>
</summary>

- Recuerde que la función `classification_report` retorna un string con el resultado. No se preocupe si no puede seleccionar una métrica en específico.
</details>

In [ ]:
# FUNCIÓN CALIFICADA evaluation:
from sklearn.metrics import classification_report

def evaluation(model, features_test, labels_test):
    ### ESCRIBA SU CÓDIGO AQUÍ ###
    report = ...
    return report
    ### FIN DEL CÓDIGO ###

In [ ]:
#TEST_CELL
features, vect = vectorizer(data.corpus)
encoded_labels, encoder = label_encode(data['review_rate'])
features_train, features_test, labels_train, labels_test = split_data(
        features, encoded_labels, test_size=0.3, seed=42
        )
model = random_forest(features_train, labels_train, 5, 50, 42)
report = evaluation(model, features_test, labels_test)
print(report)

**Salida esperada**:

En este primer ejemplo debe obtener una tabla con las métricas del modelo:

```python
❱ print(report)
              precision    recall  f1-score   support

           0       0.50      0.01      0.01       168
           1       0.00      0.00      0.00       177
           2       0.00      0.00      0.00       250
           3       0.00      0.00      0.00       248
           4       0.00      0.00      0.00       358
           5       0.19      0.79      0.30       507
           6       0.20      0.32      0.24       484
           7       0.23      0.06      0.10       418
           8       0.00      0.00      0.00       250
           9       0.00      0.00      0.00       158

    accuracy                           0.19      3018
   macro avg       0.11      0.12      0.07      3018
weighted avg       0.12      0.19      0.10      3018
```

In [ ]:
#TEST_CELL
features, vect = vectorizer(data.corpus)
encoded_labels, encoder = label_encode(data['review_rate'])
features_train, features_test, labels_train, labels_test = split_data(
        features, encoded_labels, test_size=0.4, seed=42
        )
model = random_forest(features_train, labels_train, 7, 100, 42)
report = evaluation(model, features_test, labels_test)
print(report)

**Salida esperada**:

En este segundo ejemplo debe obtener una tabla con las métricas del modelo:

```python
❱ print(report)
              precision    recall  f1-score   support

           0       0.43      0.01      0.03       223
           1       0.00      0.00      0.00       237
           2       0.00      0.00      0.00       333
           3       0.00      0.00      0.00       331
           4       0.26      0.01      0.02       477
           5       0.18      0.72      0.29       675
           6       0.20      0.33      0.25       646
           7       0.25      0.10      0.15       558
           8       0.00      0.00      0.00       333
           9       0.00      0.00      0.00       211

    accuracy                           0.19      4024
   macro avg       0.13      0.12      0.07      4024
weighted avg       0.15      0.19      0.11      4024
```

## **6. Importancia de Términos**
---

Una de las características del modelo de bosques aleatorios es que este permite extraer importancias de cada una de las características.

En este punto deberá extraer el top $N$ de términos más discriminantes de acuerdo al modelo de bosques aleatorios. Para esto, debe implementar la función `top_n_terms`, la cual recibe el vectorizador, el modelo entrenado y debe retornar una lista con los términos más relevantes.

**Parámetros**

- `vect`: vectorizador TF-IDF.
- `model`: modelo de bosques aleatorios entrenado.
- `n`: número de palabras a extraer.

**Retorna**

- `top_words`: lista con las palabras más relevantes.

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pistas</b></font>
</summary>

- Puede extraer el vocabulario del vectorizador con el método `get_feature_names_out`.
- Puede extraer la importancia de cada término del vocabulario con el atributo `feature_importances_` del modelo de bosques aleatorios.
</details>

In [ ]:
# FUNCIÓN CALIFICADA evaluation:
from sklearn.metrics import classification_report

def top_n_terms(vect, model, n):
    ### ESCRIBA SU CÓDIGO AQUÍ ###
    top_words = ...
    return top_words
    ### FIN DEL CÓDIGO ###

In [ ]:
#TEST_CELL
features, vect = vectorizer(data.corpus)
encoded_labels, encoder = label_encode(data['review_rate'])
features_train, features_test, labels_train, labels_test = split_data(
        features, encoded_labels, test_size=0.4, seed=42
        )
model = random_forest(features_train, labels_train, 7, 100, 42)
top_words = top_n_terms(vect, model, 5)
display(top_words)

**Salida esperada**:

En este ejemplo debe obtener las siguientes 5 palabras:

```python
❱ display(top_words)
['mala', 'nada', 'netflix', 'entretenida', 'bastante']
```

In [ ]:
#TEST_CELL
features, vect = vectorizer(data.corpus)
encoded_labels, encoder = label_encode(data['review_rate'])
features_train, features_test, labels_train, labels_test = split_data(
        features, encoded_labels, test_size=0.4, seed=42
        )
model = random_forest(features_train, labels_train, 5, 100, 42)
top_words = top_n_terms(vect, model, 20)
display(top_words)

**Salida esperada**:

En este ejemplo debe obtener las siguientes 20 palabras:

```python
❱ display(top_words)
['mala',
 'nada',
 'netflix',
 'bastante',
 'tampoco',
 'aburrida',
 'excelente',
 'demasiado',
 'entretenida',
 'notable',
 'bien',
 'rato',
 'interesante',
 'pero',
 'aunque',
 'correcta',
 'cada',
 'poco',
 'peor',
 'verguenza']
```

## Créditos
---

* **Profesor:** [Felipe Restrepo Calle](https://ferestrepoca.github.io/)
* **Asistentes docentes:**
    - [Juan Sebastián Lara Ramírez](https://www.linkedin.com/in/juan-sebastian-lara-ramirez-43570a214/).
* **Diseño de imágenes:**
    - [Rosa Alejandra Superlano Esquibel](mailto:rsuperlano@unal.edu.co).
* **Coordinador de virtualización:**
    - [Edder Hernández Forero](https://www.linkedin.com/in/edder-hernandez-forero-28aa8b207/).

**Uniersidad Nacional de Colombia** - *Facultad de Ingeniería*